In [26]:
import pandas as pd
import numpy as np
from IPython.display import display_html
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [27]:
df = pd.read_csv('../data/CTGsmt with titles.csv')
dataset = df.copy()
dataset.head(2)

,LBE,LB,AC,FM,UC,ASTV,MSTV,ALTV,MLTV,DL,...,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,NSP
0,0.259259,0.259259,0.000000,0.0,0.000000,0.813333,0.044118,0.472527,0.047337,0.000,...,0.110092,0.034483,0.111111,0.0,0.472441,0.587156,0.403670,0.271375,1.0,2
1,0.481481,0.481481,0.153846,0.0,0.173913,0.066667,0.279412,0.000000,0.205128,0.125,...,0.165138,0.655172,0.333333,0.1,0.637795,0.577982,0.577982,0.044610,0.5,1


In [28]:
X = dataset.copy()
X.drop(['NSP'], axis=1,  inplace=True)
Y = dataset['NSP'].copy()
X = X.to_numpy()
Y = Y.to_numpy()
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, Y, random_state=0, stratify=Y, shuffle=True)
y_test_o = y_test_d
print(X_test_d.shape)

(1242, 23)


In [29]:
def select_features(modelo, n_features, fwd, fltg):
    '''Feature Selection Function
    Recibe 4 parámetros: 1. el modelo (clf para nuestro caso), 2. el número de características final que se quiere alcanzar
    3. Si es forward (True), si es Backward False, 4. Si es es flotante (True), sino False'''
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=2,
           scoring='accuracy',
           cv=0,
           n_jobs=-1,
           pre_dispatch=10)
    
    return sfs

In [18]:
def trainSFS(n_features,clf):
    
    forward = True
    floating = False
   
        
        
    tiempo_i = time.time()
    
    #Implemetamos la metodología de validación 

    Folds = 4
    random.seed(19680801)
    f1 = np.zeros(Folds)
    auc_fpr = np.zeros(Folds)
    gmean = np.zeros(Folds)
    Errores = np.ones(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j=0
    for train_index, test_index in skf.split(X_train_d,y_train_d):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias

        #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
        #Ejercicio 3.1
        sf = select_features(clf, n_features, forward, floating)

        #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
        #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
        sf = sf.fit(X_train, y_train)

        Errores[j] = 1-sf.k_score_
        j+=1
    return  str(np.mean(Errores)), str(np.std(Errores)), str(sf.k_score_*100), str(time.time()-tiempo_i),sf.k_feature_idx_


In [19]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)
def train(clf):
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Implemetamos la metodología de validación

    j = 0
    Folds = 4
    Errores = np.ones(Folds)
    skf = StratifiedKFold(n_splits=Folds)

    for train_index, test_index in skf.split(X,Y):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entran y se valida el modelo sin hacer selección de características

        ######

        # Entrenamiento el modelo.
        model = clf.fit(X_train,y_train)

        # Validación del modelo
        ypred = model.predict(X_test)

        #######

        Errores[j] = classification_error(ypred, y_test)
        j+=1
    print('finalizado clf sin selección',str(np.mean(Errores)), str(np.std(Errores)), str((1 - np.mean(Errores))*100), str(time.time()-tiempo_i))
    return  str(np.mean(Errores)), str(np.std(Errores)), str((1 - np.mean(Errores))*100), str(time.time()-tiempo_i)

In [20]:
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['RF sin Selección','RF + SFS','RF + SFS','RF + SFS','RF + SFS','RF + SFS']),
    '# de características seleccionadas' : pd.Series([274,55,82,110,164,219]),
   })
ic_ = []
tEjec_ = []
errorVal_ = []
eficiencia_ = []
caracteristicas=[]
model=OneVsRestClassifier(RandomForestClassifier(n_estimators=500, max_features='auto', n_jobs=-1),
                          n_jobs=-1)
for e, i in zip(df_types['Tecnica'], df_types['# de características seleccionadas']):
    
    if(e == "RF sin Selección"):
        errorVal, ic, eficiencia, tEjec = train(model)
        errorVal_.append(errorVal)
        ic_.append(ic)
        tEjec_.append(tEjec+" s")
        eficiencia_.append(eficiencia)
    else:
        print('entrenando: ',e,i)
        errorVal, ic, eficiencia, tEjec,car = trainSFS(i, model)
        errorVal_.append(errorVal)
        ic_.append(ic)
        tEjec_.append(tEjec+" s")
        eficiencia_.append(eficiencia)
        caracteristicas.append(car)


NameError: name 'RandomForestClassifier' is not defined